In [1]:
import sys, os, pickle, time, json, yaml, re
from glob import glob
from prettytable import PrettyTable

In [2]:
results_dir = os.environ['HOME'] + '/public_html/BPH_RDst/Combine/'
version = '13'
cat = 'comb'

In [36]:
dirs = glob(results_dir + '*'+version+'*'+cat+'*')
auxDirs = []
for d in sorted(dirs):
    if (not os.path.basename(d).startswith('v'+version+'_')) and (not os.path.basename(d).startswith(version+'_')):
        continue
    if not 'base' in d and not 'noLostHits_mVisPM_3p8' in d:
        continue
    auxDirs.append(d)
    print d
dirs = auxDirs

/storage/af/user/ocerri/public_html/BPH_RDst/Combine/13_base_comb_CLN_blinded
/storage/af/user/ocerri/public_html/BPH_RDst/Combine/v13_noLostHits_mVisPM_3p8_CLNshape0p2_comb_CLN_blinded
/storage/af/user/ocerri/public_html/BPH_RDst/Combine/v13_noLostHits_mVisPM_3p8_CLNshapeU_comb_CLN_blinded
/storage/af/user/ocerri/public_html/BPH_RDst/Combine/v13_noLostHits_mVisPM_3p8_comb_CLN_blinded


In [37]:
dirs.pop(1)
dirs = [dirs[0], dirs[2], dirs[1]]

In [40]:
table = PrettyTable()
table.field_names = ['Version', 'Sat pval', 'Scan [%]', 'Cat comp', 'Top pulls','[sigma]']
table.align['Top pulls'] = 'l'
table.align['[sigma]'] = 'r'

for dd in dirs:
    print os.path.basename(dd)
    
    tag = os.path.basename(dd)
    tag = tag[tag.find(version+'_')+len(version+'_'):]
    tag = tag[:tag.find(cat)]
    tag = tag.replace('noLostHits_', '')
    if tag.endswith('_'):
        tag = tag[:-1]
#     print tag


    GoF_file = dd + '/GoF_results.txt'
    pval_sat = '-'
    if os.path.isfile(GoF_file):
        with open(GoF_file) as f:
            for line in reversed(f.readlines()):
                if 'algoSat' in line:
                    break
            data = [x for x in line.split(' ') if x]
            data[2] = data[2][1:-2]
            if float(data[2]) > 0:
                pval_sat = data[2] + '%'
            else:
                pval_sat = 'x{:.2f}'.format(float(data[1])/float(data[-2]))
#     print pval_sat


    scan_file = dd + '/scan_results.txt'
    scan_result = '-'
    if os.path.isfile(scan_file):
        with open(scan_file) as f:
            line = f.readlines()[-1]
            data = [x for x in line.split(' ') if x]
            err = float(data[2][1:]) + float(data[4][1:])
            scan_result = '{:.1f} +/- {:.1f}'.format(100*float(data[1]), 100*err*0.5)
#     print scan_result

    catComp_file = dd + '/categoriesCompatibility.txt'
    catComp = '-'
    if os.path.isfile(catComp_file):
        with open(catComp_file) as f:
            line = f.readlines()[-1]
            catComp = line.split(' ')[-1][1:-5] + '%'
#     print catComp

    pulls_file = dd + '/scanNuisanceOut_Base.txt'
    nPulls = 5
    pulls = ['-']*nPulls
    if os.path.isfile(pulls_file):
        with open(pulls_file) as f:
            for i, line in enumerate(f.readlines()[3:nPulls+3]):
                pulls[i] = [x for x in line[:-1].replace('|', '').split(' ') if x]
#     for p in pulls:
#         print p

    if dd != dirs[0]:
        table.add_row(len(table.field_names)*[''])
    table.add_row([tag, pval_sat, scan_result, catComp, pulls[0][0], pulls[0][1]])
    for i in range(1, len(pulls)):
        table.add_row(['', '', '', '', pulls[i][0], pulls[i][1]])

13_base_comb_CLN_blinded
v13_noLostHits_mVisPM_3p8_comb_CLN_blinded
v13_noLostHits_mVisPM_3p8_CLNshapeU_comb_CLN_blinded


In [41]:
print table

+----------------------+----------+--------------+----------+---------------------------------+---------+
|       Version        | Sat pval |   Scan [%]   | Cat comp | Top pulls                       | [sigma] |
+----------------------+----------+--------------+----------+---------------------------------+---------+
|         base         |  x1.12   | 37.9 +/- 9.1 |   71%    | B2DstCLNeig2                    |    3.17 |
|                      |          |              |          | brBu_DstDustK                   |    2.93 |
|                      |          |              |          | brBd_DstDustK                   |    2.25 |
|                      |          |              |          | prop_binhigh_ctrl_pm_mVis_bin15 |   -2.11 |
|                      |          |              |          | brDst2S_DstPiPi                 |    2.07 |
|                      |          |              |          |                                 |         |
|      mVisPM_3p8      |   8.2%   | 43.1 +/- 9

In [ ]:
# outfile = results_dir + 'summary_table_{}_{}.txt'.format(version, cat)
# os.system('date > '+outfile)
# with open(outfile, 'a') as f:
#     f.write(table.get_string())
    
# with open(outfile[:-4]+'.html', 'w') as f:
#     f.write(table.get_html_string())